Homework 1

Royce Lloyd, 2/22/2019, for CS344 at Calvin College

Professor Vander Linden

1.1  It seems to me that AI would have to use some form of API to run on itself to determine what it is doing and display that output. This increases the focus on the "intelligence" part of the AI, which by its own nature, it does not contain like a human does. If we wanted AI to think about itself, we would have to be prepared for the existential repercussions of AI that can have honest "feelings". To me, I'd prefer for it to be a long series of complicated instructions that only mimic human ability. 
I would assume that an Ai with these capabilities could be effectively very beneficial for the bottom line. A robot with the ability to know how import its job is might, figuratively, try harder. That statement, however, is not consistent with what I as a christian believe can be introspective. Thinking about yourself is special to creatures made in the image of God. That alone is humans, given dominion over creation, which includes computer technology. 

In [1]:
"""
This file solves the TSP problem using Hill Climbing and SA.
It can be configured with random distances and number of cities

Written by Royce Lloyd for CS 344 at Calvin College
2/22/2019
"""

from search import Problem, hill_climbing, simulated_annealing, \
    exp_schedule, genetic_search
from random import randrange
import math


class TSP(Problem):
    """
    State: x value for the abs function variant f(x)
    Move: a new x value delta steps from the current x (in both directions)
    """

    
    def __init__(self, numCities):
        self.numCities = numCities
        self.citiesDistances = []
        self.distances = []
        self.distancesMatrix = []
        

        self.initial = []
        
        #Create an array with the numeric identifies of each city
        for x in range(0, numCities):
            self.initial.append(x)

        """
        This was the best way I found to create a bunch of random distance values that was guaranteed to have
        enough values for each city
        """

        decrement = 1
        #Creates a path to each city, starting at city 0
        #City zero in a city of 4, four example, has 3 paths
        #City one will have 2 paths, since it's path to city zero has already been established
        #Continue to decrement the number of new paths as you continue through each city. 
        for city in range(numCities):
            for paths in range(numCities - decrement):
                value = randrange(25, 100)
                self.distances.append(value)
            decrement += 1

        
        #Fill an array with the distances avaliable
        #Use these confirmed distances to fill the beginnings of a matrix
        for city in range(0, numCities):
            cityArray = []
            for x in range(0, numCities ):
                cityArray.append(self.distances[randrange(0, len(self.distances))])
            self.distancesMatrix.append(cityArray)


        #distancesMatrix is a maping for each distance
        #Example, the point (0,2) is the distanct from city 0 to 2, and vice versa
        #This loop mirrors the matrix over the diagonal
        for x in range(0, numCities):
            for city in range(0, numCities):
                self.distancesMatrix[x][city] = self.distancesMatrix[city][x]

    #Create a list of actions for each algorithm to choose
    def actions(self, state):
        idea_list = []
        
        for x in range(0, self.numCities):
            for y in range(0, self.numCities):
                if(x != y):
                    idea = [x, y]
                    idea_list.append(idea)
        return idea_list

    #use the results of the action step to switch the order of the final array of cities
    def result(self, state, action):
        state_copy = state[:]
        temp_state = state_copy[action[0]]
        state_copy[action[0]] = state_copy[action[1]]
        state_copy[action[1]] = temp_state
        return state_copy


    #Determine the value of the shortest distance to travel
    def value(self, state):
        sum = 0

        for x in range(0, len(state)):
            cityA = state[x]
            cityB = state[(x + 1) % len(state)]
            sum += self.distancesMatrix[cityA][cityB]
        #Because we want the smallest distance, we will create some large number to subtract the sum from
        maxDistance = 100 * numCities
        sum = maxDistance - sum
        return sum


if __name__ == '__main__':

    numCities = 25

    p = TSP(numCities)

    #Solve the problem using hill-climbing.
    hill_solution = hill_climbing(p)
    print(str(hill_solution))
    print(p.value(hill_solution))

    #Solve the problem with SA
    annealing_solution = simulated_annealing(
        p,
        exp_schedule(k=20, lam=0.005, limit=1000)
    )
    print(str(annealing_solution))
    print(p.value(annealing_solution))

[8, 9, 2, 5, 6, 1, 4, 22, 0, 3, 23, 20, 12, 14, 16, 15, 13, 11, 18, 19, 17, 21, 7, 10, 24]
1632


[20, 15, 5, 6, 16, 13, 17, 3, 22, 10, 24, 11, 1, 18, 2, 9, 0, 8, 19, 4, 14, 21, 7, 23, 12]
1691


In [ ]:
"""
This module schedules a list of CS classes with times, a room, and most importantly, a professor.
It will check of conflicts using CSP and adjust accordingly.  

Written by Royce Lloyd for CS 344 at Calvin College
2/22/2019
"""


from csp import backtracking_search, parse_neighbors, CSP, min_conflicts, AC3

def scheduler():

    classes = "CS232 CS108 CS212 CS112 CS336 CS344".split()
    profs = "Adams VanderLinden Plantinga Schuurman Norman Nelsen".split()
    times = "mwf0800 mwf0900 mwf1000 th1230".split()
    rooms = "nh253 sb382".split()

    #Because each class can only be taught once at a time, it is the best variable 
    variables = classes

    #Fill an array with every possible domain
    all_domains = []
    for room in rooms:
        for time in times:
            for prof in profs:
                all_domains.append(room + " " + time + " " + prof)

    #Each domain is created based on the variables
    domains = {}
    for var in variables:
        domains[var] = all_domains

    #If a variable is changed, changed its neighboring variabels accordingly 
    neighbors = parse_neighbors("""CS344: nh253 mwf0800""", variables)
    for type in [variables]:
        for A in type:
            for B in type:
                if A != B:
                    if B not in neighbors[A]:
                        neighbors[A].append(B)
                    if A not in neighbors[B]:
                        neighbors[B].append(A)

    def scheduler_constraint(A, a, B, b, recurse=0):

        conflict = (a == b)

        #a at index 0 is prof
        #index 1 is room
        #index 2 is time
        if(a.split()[0] == b.split()[0]) and (a.split()[2] == b.split()[2]):
            return conflict

        if(a.split()[1] == b.split()[1]) and (a.split()[2] == b.split()[2]):
            return conflict

        if recurse == 0:
            return scheduler_constraint(B, b, A, a, 1)

        return True

    return CSP(variables, domains, neighbors, scheduler_constraint)


#Uncomment each method as you like to se the results for each algorithm

#print(backtracking_search(scheduler()))
#print(AC3(scheduler()))
#print(min_conflicts(scheduler()))

For the TSP problem: It seems that SA is the most consistent in getting the correct answer. However, hill climbing for lower values is less likely to get caught in a local maximum, and performs slightly better. I would most often go for SA.
